In [2]:
# This is how I applied a binary mask on the images then from the build the dataset 
# Bad coding practice I know but just comment out training once your done generating the mask and move on to uncommenting test and so on

import cv2
import os

# For training masking 
input_dir = 'train/images/'
output_dir = 'Annotated/train'

#For test masking
#input_dir = 'test/images/'
#output_dir = 'Annotated/test'

#For valid masking
#input_dir = '/images/'
#output_dir = 'Annotated/valid'


# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Full path to the input image
        image_path = os.path.join(input_dir, filename)

        # Read the image
        image = cv2.imread(image_path)

        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Apply thresholding to create a binary mask
        _, binary_mask = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        # Create the output file path
        output_path = os.path.join(output_dir, filename)

        # Save the binary mask to the output directory
        cv2.imwrite(output_path, binary_mask)



In [4]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('cloud.png', cv2.IMREAD_GRAYSCALE)

# Define thresholds
threshold1 = 50
threshold2 = 150

# Apply thresholding
_, thresh1 = cv2.threshold(image, threshold1, 255, cv2.THRESH_BINARY)
_, thresh2 = cv2.threshold(image, threshold2, 255, cv2.THRESH_BINARY)

# Create an image to combine the thresholds
thresh_combined = np.zeros_like(image)

# Set pixels based on thresholds
thresh_combined[(image <= threshold1)] = 0
thresh_combined[(image > threshold1) & (image <= threshold2)] = 127
thresh_combined[(image > threshold2)] = 255

# Save or display the result
cv2.imwrite('thresholded_image.png', thresh_combined)
cv2.imshow('Thresholded Image', thresh_combined)
cv2.waitKey(0)
cv2.destroyAllWindows()


TypeError: '<=' not supported between instances of 'NoneType' and 'int'